In [1]:
#importando librerias en general
import pandas as pd
import numpy as np
from time import time

In [107]:
#cargando datasets
dataPCAtrain = pd.read_csv("PCA_train.csv") #datos en el espacio de las pca
dataNormaltrain = pd.read_csv("train.csv")
ytrain = np.ravel(pd.read_csv("train_labels.csv").apply(lambda x: 1 if x > 5 else 0).values) #variable respuesta

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [13]:
ytrain = np.array(list(map(lambda x: 1 if x > 5 else 0, ytrain)))
ytrain

array([0, 0, 0, ..., 1, 0, 1])

In [111]:
#importing sklearn modules
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV

In [19]:
param_grid = {"n_estimators": ([2**i for i in range(6)] + [100, 200]),
    "max_depth": (list(range(5, 11)) + [25, 50]),
    "max_leaf_nodes": [4,5,6,7,8],
    "max_features": ['auto', 'sqrt', 'log2'],
    "criterion": ['gini', 'entropy']
}

In [62]:
rf_PCA = RandomForestClassifier(random_state=42)
t1 = time()
gs_PCA = GridSearchCV(rf_PCA, param_grid, cv = 3, scoring = "accuracy", return_train_score= True)
gs_PCA.fit(dataPCAtrain, ytrain)
t1f = time() - t1

In [63]:
mean_time_PCA = gs_PCA.cv_results_['mean_fit_time']
mean_score_test_PCA = gs_PCA.cv_results_['mean_test_score']
mean_score_train_PCA = gs_PCA.cv_results_['mean_train_score']
best_score = gs_PCA.best_score_

In [64]:
rf_Norm = RandomForestClassifier(random_state=42)
t2 = time()
gs_Norm = GridSearchCV(rf_Norm, param_grid, cv = 3, scoring = "accuracy", return_train_score= True)
gs_Norm.fit(dataNormaltrain, ytrain)
t2f = time() - t2

In [65]:
mean_time_Norm = gs_Norm.cv_results_['mean_fit_time']
mean_score_test_Norm = gs_Norm.cv_results_['mean_test_score']
mean_score_train_Norm = gs_Norm.cv_results_['mean_train_score']
best_score_Norm = gs_Norm.best_score_

In [67]:
list_PCA = {"mean_time": mean_time_Norm, "mean_score_test": mean_score_test_PCA,
    "mean_score_train": mean_score_train_PCA}
list_Norm = {"mean_time": mean_time_PCA, "mean_score_test": mean_score_test_Norm,
    "mean_score_train": mean_score_train_Norm}

In [73]:
df_PCA = pd.DataFrame(list_PCA)
df_Norm = pd.DataFrame(list_Norm)

In [74]:
df_PCA.to_csv("DatoscompPCA.csv")
df_Norm.to_csv("DatoscompNorm.csv")

In [79]:
mod_PCA = gs_PCA.best_estimator_
mod_Norm = gs_Norm.best_estimator_

In [98]:
ytrain = ytrain.reshape((1279, 1))

In [101]:
PCA_predict = mod_PCA.predict(dataPCAtrain)
conf_PCA = confusion_matrix(ytrain, PCA_predict)

In [102]:
Norm_predict = mod_Norm.predict(dataNormaltrain)
conf_Norm = confusion_matrix(ytrain, Norm_predict)

In [108]:
test_Norm = pd.read_csv("test_wine.csv")
test_labels = np.array(list(map(lambda x: 1 if x > 5 else 0, pd.read_csv("test_labels.csv").values))).reshape((320,1))

In [109]:
predicttest_Norm = mod_Norm.predict(test_Norm)
conf_Norm_test = confusion_matrix(test_labels, predicttest_Norm)
conf_Norm_test

array([[160,   0],
       [  0, 160]])

In [110]:
test_pca = pd.read_csv("test_pca.csv")
predicttest_PCA = mod_PCA.predict(test_pca)
conf_PCA_test = confusion_matrix(test_labels, predicttest_PCA)
conf_PCA_test

array([[149,  11],
       [ 19, 141]])

In [115]:
accuracy_score(test_labels, predicttest_Norm), precision_score(test_labels, predicttest_Norm) ,recall_score(test_labels, predicttest_Norm)


(1.0, 1.0, 1.0)

In [116]:
accuracy_score(test_labels, predicttest_PCA), precision_score(test_labels, predicttest_PCA) ,recall_score(test_labels, predicttest_PCA)

(0.90625, 0.9276315789473685, 0.88125)